In [2]:
import pandas as pd
csv_path = '../csv/BTC/KRW_BTC_month.csv'
df = pd.read_csv(csv_path)

df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.set_index('Unnamed: 0', inplace=True)
df.columns = df.columns.str.capitalize()

In [6]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

def RSI(values, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(values).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

class RsiCross(Strategy):
    lower_bound = 30
    upper_bound = 70
    lookback_period = 14

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.lookback_period)

    def next(self):
        if crossover(self.lower_bound, self.rsi):
            self.position.close()
            self.buy()
        elif crossover(self.rsi, self.upper_bound):
            self.position.close()
            self.sell()
bt = Backtest(df, RsiCross, cash=1000000000, commission=0.002)
stats = bt.run()
stats



Start                     2023-05-02 09:00:00
End                       2024-05-23 09:00:00
Duration                    387 days 00:00:00
Exposure Time [%]                   98.969072
Equity Final [$]                  609956298.0
Equity Peak [$]                  1342405524.0
Return [%]                          -39.00437
Buy & Hold Return [%]              147.103242
Return (Ann.) [%]                  -37.190425
Volatility (Ann.) [%]               47.910475
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -76.715435
Avg. Drawdown [%]                   -6.425826
Max. Drawdown Duration      216 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   15
Win Rate [%]                        33.333333
Best Trade [%]                      22.912898
Worst Trade [%]                    -31.381446
Avg. Trade [%]                    